# Chatbot Part1: Pre-Model

## Import Libraries

In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random 
import pandas as pd

## Setting variables

In [2]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('resources/commands.json').read()
intents = json.loads(data_file)

## Data actions

In [3]:
# Data pre-processing
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #print (pattern)
        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag'])) # e.g. [(['Hi', 'there'], 'greeting')]
        #print (documents)
        
        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [4]:
classes

['greeting',
 'goodbye',
 'thanks',
 'options',
 'adverse_drug',
 'blood_pressure',
 'blood_pressure_search',
 'pharmacy_search',
 'hospital_search']

In [5]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents" , documents)
print ('\n')
print (len(classes), "classes", classes)
print ('\n')
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('resources/words.pkl','wb'))
pickle.dump(classes,open('resources/classes.pkl','wb'))

47 documents [(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'), (['Till', 'next', 'time'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Awesome', ',', 'thanks'], 'thanks'), (['Thanks', 'for', 'helping', 'me'], 'thanks'), (['How', 'you', 'could', 'help', 'me', '?'], 'options'), (['What', 'you', 'can', 'do', '?'], 'options'), (['What', 'help', 'you', 'provide', '?'], 'options'), (['How', 'you', 'can', 'be', 'helpful', '?'], 'options'), (['What', 'support', 'is', 'offered'], 'options'), (['How', 'to', 'check', 'Adverse', 'drug', 'reaction', '?'], 'adverse_drug'), (['Open', 'adverse', 'drugs', 'module'], 'adverse_drug'),

# Chatbot Part 2: Model

## Building model

In [6]:
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pd.Series(pattern_words).str.lower()]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


<ipython-input-6-837c43a794e5>:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [7]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/200
10/10 [==============================] - 0s 1ms/step - loss: 2.2823 - accuracy: 0.0638
Epoch 2/200
10/10 [==============================] - 0s 1ms/step - loss: 2.1180 - accuracy: 0.1277
Epoch 3/200
10/10 [==============================] - 0s 1ms/step - loss: 2.0658 - accuracy: 0.2553
Epoch 4/200
10/10 [==============================] - 0s 1ms/step - loss: 1.9704 - accuracy: 0.3830
Epoch 5/200
10/10 [==============================] - 0s 1ms/step - loss: 1.8593 - accuracy: 0.4255
Epoch 6/200
10/10 [==============================] - 0s 1ms/step - loss: 1.7616 - accuracy: 0.4681
Epoch 7/200
10/10 [==============================] - 0s 1ms/step - loss: 1.7090 - accuracy: 0.5106
Epoch 8/200
10/10 [==============================] - 0s 1ms/step - loss: 1.5893 - accuracy: 0.5745
Epoch 9/200
10/10 [==============================] - 0s 1ms/step - loss: 1.3844 - accuracy: 0.5957
Epoch 10/200
10/10 [==============================] - 0s 1ms/step - loss: 1.1848 - accuracy: 0.6809
Epoch 11/

Epoch 83/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0571 - accuracy: 1.0000
Epoch 84/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0520 - accuracy: 1.0000
Epoch 85/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0805 - accuracy: 0.9574
Epoch 86/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0115 - accuracy: 1.0000
Epoch 87/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0270 - accuracy: 1.0000
Epoch 88/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0734 - accuracy: 0.9787
Epoch 89/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0458 - accuracy: 0.9787
Epoch 90/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0292 - accuracy: 1.0000
Epoch 91/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0516 - accuracy: 1.0000
Epoch 92/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0227 - accuracy: 1.0000


10/10 [==============================] - 0s 1ms/step - loss: 0.0112 - accuracy: 1.0000
Epoch 165/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0529 - accuracy: 0.9787
Epoch 166/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0054 - accuracy: 1.0000
Epoch 167/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0040 - accuracy: 1.0000
Epoch 168/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0073 - accuracy: 1.0000
Epoch 169/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0354 - accuracy: 0.9787
Epoch 170/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0150 - accuracy: 1.0000
Epoch 171/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0049 - accuracy: 1.0000
Epoch 172/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0034 - accuracy: 1.0000
Epoch 173/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0790 - accuracy: 0.9362
Epoc

In [8]:
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('resources/commands.json').read())
words = pickle.load(open('resources/words.pkl','rb'))
classes = pickle.load(open('resources/classes.pkl','rb'))

## Define necessary functions

In [9]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    #print (p)
    res = model.predict(np.array([p]))[0]
    #print (res)
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    #print (ints)
    res = getResponse(ints, intents)
    return res

## Makin' That Chatbot!

In [10]:
while True:
    query = input("User (enter quit() to exit the bot) > ")
    query1 = [chatbot_response(query)]
    question = " ".join(query1)
    response = question #k.respond(question)
    if (query=='quit()'):
        print ('bot > Good Bye, see you again !!!')
        break
    
    if response:
        print("bot > ", response)
    else:
        print("bot > :) ", )

User (enter quit() to exit the bot) > pitu
bot >  Hello, thanks for asking
User (enter quit() to exit the bot) > pitu
bot >  Good to see you again
User (enter quit() to exit the bot) > I NEED HOSPITAL
bot >  Please provide hospital name or location
User (enter quit() to exit the bot) > WHO IS SPIDERMAN
bot >  Hi there, how can I help?
User (enter quit() to exit the bot) > Give me pharmacy
bot >  Please provide pharmacy name
User (enter quit() to exit the bot) > dshjksdfhjgkrkejgkjdfshjgkfjhsfdhg pharmacy hjdfghkjdsfhjhdfkgjkhdfjskg
bot >  Please provide pharmacy name
User (enter quit() to exit the bot) > quit()
bot > Good Bye, see you again !!!
